# Vapor liquid Equilibria: Bubble and dew Points

The isothermal-isobaric two-phase flash is the base for the calculation Vapor-Liquid Equilibria. This calculation is based on the solution of the Rachford-Rice mass balance. 

$$ FO = \sum_{i=1}^c \left( x_i^\beta - x_i^\alpha \right) = \sum_{i=1}^c \frac{z_i (K_i-1)}{1+\psi (K_i-1)} $$


Where,  $K_i = x_i^\beta/x_i^\alpha =\hat{\phi}_i^\alpha /\hat{\phi}_i^\beta  $ represents the equilibrium constant  and $\psi$ the fraction of the phase $\beta$. For bubble and dew points calculations the phase fraction $\psi$ is known beforehand and set to 0 for bubble points (differential size bubble) and to 1 for dew point (differential size liquid drop).

The Rachford-Rice mass balance reduces to the following equations:

### Bubble

$$ FO = \sum_{i=1}^c x_i (K_i-1) = \sum_{i=1}^c y_i -1 = 0 $$


###  Dew 

$$ FO = 1 - \sum_{i=1}^c \frac{y_i}{K_i} = 1 - \sum_{i=1}^c x_i = 0 $$


The solution of these calculations includes using accelerated successive substitution (ASS) to update the phase compositions in an inner loop and the quasi-Newton method is used to update pressure or temperature in an outer loop. If slow convergence is detected, the algorithm attempts to solve the following system of equations using equilibrium constants, $K$, as iteration variables. This is done using SciPy's optimization routines.

$$ f_i = \ln K_i + \ln \hat{\phi}_i^v(\underline{y}, T, P) -\ln \hat{\phi}_i^l(\underline{x}, T, P) \quad i = 1,...,c $$
$$ f_{c+1} = \sum_{i=1}^c (y_i-x_i) $$


**note:** these calculations does not check for the stability of the phases.

In [1]:
import numpy as np
from SGTPy import component, mixture, saftvrmie
from SGTPy.equilibrium import bubblePy, bubbleTy
from SGTPy.equilibrium import dewPx, dewTx

---
### Bubble point calculation

This calculation will be exemplified for the mixture of hexane and ethanol.

In [2]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

hexane = component('hexane', ms = 1.96720036, sigma = 4.54762477, eps = 377.60127994, 
                   lambda_r = 18.41193194, cii = 3.581510586936205e-19)

mix = mixture(hexane, ethanol)
# fitted to experimental data
kij = 0.011818492037463553
Kij = np.array([[0, kij], [kij, 0]])
mix.kij_saft(Kij)
eos = saftvrmie(mix)

#### Bubble point algorithm x, P -> y, T

In [3]:
# bubble point conditions
P = 1.01325e5  # Pa
x = np.array([0.2, 0.8])
# initial guess for temperature and vapor composition
T0 = 320.
y0 = np.array([0.8, 0.2])
# vapor composition and equilibrium temperature
bubbleTy(y0, T0, x, P, eos)

(array([0.58026271, 0.41973729]), 333.45266622855047)

#### Bubble point algorithm x, T -> y, P

In [4]:
# bubble point conditions
T = 350.  # K
x = np.array([0.2, 0.8])
# initial guess for tempertature and vapor composition
P0 = 1e5 # Pa
y0 = np.array([0.8, 0.2])
# vapor composition and equilibrium pressure
bubblePy(y0, P0, x, T, eos)

(array([0.52007489, 0.47992511]), 178462.05087828575)

---
### Dew point calculation

This calculation will be exemplified for the mixture of ethanol and cpme.

In [5]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

cpme = component('cpme', ms =  2.32521144, sigma = 4.13606074, eps = 343.91193798, lambda_r = 14.15484877, 
                 lambda_a = 6.0, npol = 1.91990385,mupol = 1.27, sites =[0,0,1], cii = 3.5213681817448466e-19)

mix = mixture(ethanol, cpme)

kij = 0.01015194

Kij = np.array([[0, kij], [kij, 0]])
mix.kij_saft(Kij)
eos = saftvrmie(mix)

# induced association set up
rc = 2.23153033 # Angstrom
eos.eABij[0,1] = ethanol.eAB / 2
eos.eABij[1,0] = ethanol.eAB / 2
eos.rcij[0,1] = rc * 1e-10
eos.rcij[1,0] = rc * 1e-10


#### Dew point algorithm y, P -> x, T

In [6]:
# dew point conditions
P = 1.01325e5  # Pa
y = np.array([0.4, 0.6])
# initial guess for temperature and liquid composition
T0 = 350.
x0 = np.array([0.2, 0.8])
# liquid composition and equilibrium temperature
dewTx(x0, T0, y, P, eos)

(array([0.10611088, 0.89388912]), 364.3596382467786)

#### Dew point algorithm y, T -> x, P

In [7]:
# dew point conditions
T = 350. # K
y = np.array([0.4, 0.6])
# initial guess for temperature and liquid composition
P0 = 1e5 # Pa
x0 = np.array([0.2, 0.8])
# liquid composition and equilibrium pressure
dewPx(x0, P0, y, T, eos)


(array([0.10431591, 0.89568409]), 62927.00599008181)

Optionally, for any of these VLE computation functions you can supply initial guesses for the phase volumes (``v0``) or the non-bonded association sites solution (``Xass0``) for any of these functions. These initial guesses can be obtained from previous computations and the ``full_output=True`` option.

In [8]:
sol_dew = dewPx(x0, P0, y, T, eos, full_output=True)
sol_dew

      T: 350.0
      P: 62927.00599008181
  error: 2.2959412149248237e-13
   iter: 4
      X: array([0.10431591, 0.89568409])
     v1: 0.0001175032768988384
  Xassl: array([0.10966821, 0.76435795, 0.92768912])
 state1: 'Liquid'
      Y: array([0.4, 0.6])
     v2: 0.045319291666893455
  Xassv: array([0.97993889, 0.99147469, 0.99792689])
 state2: 'Vapor'

In [9]:
# VLE supplying initial guess for volumes and non-bonded association sites fractions
v0 = [sol_dew.v1, sol_dew.v2]
Xass0 = [sol_dew.Xassl, sol_dew.Xassv]
dewPx(x0, P0, y, T, eos, v0=v0, Xass0=Xass0, full_output=True)

      T: 350.0
      P: 62927.00599008251
  error: 4.303224443447107e-13
   iter: 4
      X: array([0.10431591, 0.89568409])
     v1: 0.00011750327689976686
  Xassl: array([0.10966821, 0.76435795, 0.92768912])
 state1: 'Liquid'
      Y: array([0.4, 0.6])
     v2: 0.045319291666883595
  Xassv: array([0.97993889, 0.99147469, 0.99792689])
 state2: 'Vapor'

---
For further information about each function check out the documentation running: ``function?``